In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from merlion.utils.time_series import TimeSeries as merlion_timeSeries
from omnixai.data.timeseries import Timeseries as omnixai_timeSeries

from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from omnixai.explainers.timeseries.agnostic.shap import ShapTimeseries

In [2]:
#Read Data
url='https://raw.githubusercontent.com/AmirMK/XAI_Time_Seris_SF/main/my_test_data.csv'
df = pd.read_csv(url)
time = df['Month of Datetime'].tolist()
target = df['#Passengers'].tolist() 

#Dara Pre-Processing
time = [datetime.strptime(date[:10],'%Y-%m-%d').date() for date in time]
df = pd.DataFrame(
        {'Datetime': time,
         'Target': target
        })
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)
df.index.freq = 'MS'

In [3]:
#Train data with merlion_timeSeries as merlion is used to build the model
train_data = df[~df['Target'].isna()]
train_data = merlion_timeSeries.from_pd(train_data)

#Test data with omnixai_timeSeries as omnixai is used to explain the model
test_data = df[df['Target'].isna()]
test_data['Target']=1
test_data = omnixai_timeSeries.from_pd(test_data)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
#Creat and train the model
model_config = DefaultForecasterConfig()
model = DefaultForecaster(model_config)

train = model.train(train_data)

In [5]:
#forecasting function accept omnixai_timeSeries

def forecasting_function(time_stamps):        
    df = time_stamps.to_pd().reset_index()        
    scores = []
    for x in df.values:
        scores = model.forecast(x[1])[0].to_pd().reset_index()['Target']
        scores.append(scores / x.shape[0])
    
    return np.array(scores)

In [6]:
forecasting_function(test_data)

array([451.35407509])

In [7]:
#Creat shap explnaier 
explainers = ShapTimeseries(
    training_data=omnixai_timeSeries.from_pd(train_data.to_pd()), #train data as merlion_timeSeries converted to omnixai_timeSeries 
    predict_function =forecasting_function,
    mode ='forecasting'
)

In [8]:
explanations = explainers.explain(test_data)

Provided model function fails when applied to the provided data set.


AttributeError: 'list' object has no attribute 'reset_index'